In [1]:
import openai

In [ ]:
from openai import OpenAI
import os
#print(os.getenv("OPENAI_API_KEY"))
os.environ

In [4]:
client = OpenAI()

In [6]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "Is it too late to join the llm_zoomcamp course?" }]
)

In [8]:
response.choices[0].message.content

"As of my last update in October 2023, I don't have specific information about the current status or registration deadlines for the LLM (Large Language Model) Zoomcamp course. The timing and availability for joining such courses can vary depending on the organization offering them.\n\nTo find out if it's too late to join the LLM Zoomcamp course, you should:\n\n1. **Check the Official Website or Communication Channels**: Look for any announcements or updates regarding enrollment deadlines or late registration policies.\n\n2. **Contact the Organizers**: Reach out directly to the organizers or facilitators of the course. They can provide the most accurate and up-to-date information regarding availability and registration.\n\n3. **Look for Community Discussions**: If there's a community forum or social media group related to the course, it might have information from other participants or updates from the organizers.\n\nIf you provide more specific details about the course provider, I migh